In [1]:
import h2o
h2o.init()
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_172"; Java(TM) SE Runtime Environment (build 1.8.0_172-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.172-b11, mixed mode)
  Starting server from /anaconda2/envs/spike_basico/h2o_jar/h2o.jar
  Ice root: /var/folders/3z/5qxln1rj4zjblz0l0hvy0n540000gn/T/tmpjed6i27v
  JVM stdout: /var/folders/3z/5qxln1rj4zjblz0l0hvy0n540000gn/T/tmpjed6i27v/h2o_maravenag_started_from_python.out
  JVM stderr: /var/folders/3z/5qxln1rj4zjblz0l0hvy0n540000gn/T/tmpjed6i27v/h2o_maravenag_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Santiago
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,7 months and 4 days !!!
H2O cluster name:,H2O_from_python_maravenag_lbq9r3
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [2]:
data = h2o.upload_file("cereal.csv", destination_frame="train")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
data = data.drop("name") #TODO w2v of names

In [4]:
data.describe()

Rows:77
Cols:15




,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
type,enum,enum,int,int,int,int,real,real,int,int,int,int,real,real,real
mins,,,50.0,1.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,1.0,0.5,0.25,18.042851
mean,,,106.8831168831169,2.545454545454545,1.0129870129870129,159.67532467532473,2.1519480519480525,14.597402597402601,6.922077922077923,96.07792207792208,28.246753246753244,2.2077922077922074,1.0296103896103896,0.8210389610389608,42.665704987012965
maxs,,,160.0,6.0,5.0,320.0,14.0,23.0,15.0,330.0,100.0,3.0,1.5,1.5,93.704912
sigma,,,19.484119056820845,1.094789748445534,1.0064725594803927,83.83229524009316,2.3833639643872226,4.278956280325907,4.4448853924193585,71.2868125092621,22.342522500566307,0.832524100135788,0.15047679973689213,0.23271613844691388,14.047288743735217
zeros,,,0,0,27,9,19,0,7,0,8,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,N,C,70.0,4.0,1.0,130.0,10.0,5.0,6.0,280.0,25.0,3.0,1.0,0.33,68.402973
1,Q,C,120.0,3.0,5.0,15.0,2.0,8.0,8.0,135.0,0.0,3.0,1.0,1.0,33.983679
2,K,C,70.0,4.0,1.0,260.0,9.0,7.0,5.0,320.0,25.0,3.0,1.0,0.33,59.425505


In [5]:
data['mfr'] = data['mfr'].asfactor()
data['type'] = data['type'].asfactor()

In [6]:
target = "rating"

In [7]:
train = data

In [8]:
train_cols = [x for x in train.col_names if x not in ['rating']]

In [30]:
from h2o.estimators import H2OXGBoostEstimator
param = {
      "ntrees" : 1000
    , "max_depth" : 5
    , "learn_rate" : 0.01
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
    , "nfolds": 4
    }

model = H2OXGBoostEstimator(**param)
model.train(x=train_cols,y=target,training_frame=train)
model.summary

xgboost Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1539203830115_12


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 3.0463850192816184
RMSE: 1.7453896468358057
MAE: 0.7456626892089844
RMSLE: 0.025406408640985898
Mean Residual Deviance: 3.0463850192816184

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 35.38911416734118
RMSE: 5.948875033764046
MAE: 3.7132217853100267
RMSLE: 0.11046380079652524
Mean Residual Deviance: 35.38911416734118
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid
mae,1.4843518,0.1765503,1.3779463,1.8832344,1.4725734,1.2036527
mean_residual_deviance,11.088014,5.4814773,6.9769115,24.506191,6.6724415,6.1965103
mse,11.088014,5.4814773,6.9769115,24.506191,6.6724415,6.1965103
r2,0.9430694,0.0281443,0.9641776,0.8741747,0.9657408,0.9681845
residual_deviance,11.088014,5.4814773,6.9769115,24.506191,6.6724415,6.1965103
rmse,3.166036,0.7294624,2.6413844,4.9503727,2.583107,2.489279
rmsle,0.0580605,0.0100610,0.0555494,0.0820050,0.0468680,0.0478196


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-10-10 17:48:10,7.913 sec,0.0,44.4152036,42.1657050,1972.7103148
,2018-10-10 17:48:10,8.073 sec,100.0,16.9590856,15.4659586,287.6105859
,2018-10-10 17:48:11,8.270 sec,200.0,7.1816429,5.7562014,51.5759954
,2018-10-10 17:48:11,8.507 sec,300.0,3.9815551,2.5291158,15.8527810
,2018-10-10 17:48:11,8.806 sec,400.0,2.9242516,1.5058519,8.5512477
,2018-10-10 17:48:11,9.172 sec,500.0,2.5279305,1.2179718,6.3904325
,2018-10-10 17:48:12,9.563 sec,600.0,2.3056938,1.0842452,5.3162240
,2018-10-10 17:48:12,10.000 sec,700.0,2.1387714,0.9792090,4.5743430
,2018-10-10 17:48:13,10.494 sec,800.0,1.9951047,0.8890898,3.9804427
,2018-10-10 17:48:13,11.026 sec,900.0,1.8657954,0.8156268,3.4811924


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
sugars,1127.0,1.0,0.1706800
sodium,932.0,0.8269743,0.1411480
potass,779.0,0.6912156,0.1179767
carbo,744.0,0.6601597,0.1126761
fiber,592.0,0.5252884,0.0896562
calories,486.0,0.4312334,0.0736029
protein,432.0,0.3833185,0.0654248
fat,381.0,0.3380657,0.0577010
cups,374.0,0.3318545,0.0566409
shelf,226.0,0.2005324,0.0342269


<bound method ModelBase.summary of >

In [31]:
### using autoencoder for the data

In [32]:
from h2o.estimators import H2OAutoEncoderEstimator

autoencoder = H2OAutoEncoderEstimator(activation="Tanh",
                              hidden=[25],
                              ignore_const_cols=False,
                              epochs=2000)

autoencoder.train(x=train_cols, training_frame=data)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [33]:
autoencoder._model_json["output"]["training_metrics"]


ModelMetricsAutoEncoder: deeplearning
** Reported on train data. **

MSE: 0.0002812971503097929
RMSE: 0.016771915522974495


In [34]:
train = data

In [35]:
train_deep_feats = autoencoder.predict(train)
train_deep_feats = train_deep_feats.cbind(train['rating'])

deep_cols = [x for x in train_deep_feats.col_names if x not in ['rating']]

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [36]:
from h2o.estimators import H2OXGBoostEstimator
param = {
      "ntrees" : 1000
    , "max_depth" : 5
    , "learn_rate" : 0.01
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
    , "nfolds": 4
    }

model = H2OXGBoostEstimator(**param)
model.train(x=deep_cols,y=target,training_frame=train_deep_feats)
model.summary

xgboost Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1539203830115_14


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 2.2500614881919554
RMSE: 1.5000204959239576
MAE: 0.47433169476397624
RMSLE: 0.020054750937468543
Mean Residual Deviance: 2.2500614881919554

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 38.56179252843272
RMSE: 6.209814210460142
MAE: 4.110553642372032
RMSLE: 0.11605827100477872
Mean Residual Deviance: 38.56179252843272
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid
mae,1.375283,0.1585036,1.3389386,1.5866401,1.552049,1.0235044
mean_residual_deviance,11.279733,4.900776,7.72362,23.072931,8.962655,5.359724
mse,11.279733,4.900776,7.72362,23.072931,8.962655,5.359724
r2,0.942085,0.0251627,0.9603436,0.8815337,0.9539819,0.9724809
residual_deviance,11.279733,4.900776,7.72362,23.072931,8.962655,5.359724
rmse,3.2228615,0.6681675,2.77914,4.803429,2.9937694,2.3151078
rmsle,0.0599288,0.0072478,0.0610392,0.0756677,0.0553353,0.0476732


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-10-10 17:48:33,8.150 sec,0.0,44.4152036,42.1657050,1972.7103148
,2018-10-10 17:48:34,8.344 sec,100.0,16.8898243,15.4771306,285.2661641
,2018-10-10 17:48:34,8.564 sec,200.0,6.9915813,5.7036752,48.8822090
,2018-10-10 17:48:34,8.822 sec,300.0,3.7364926,2.3019989,13.9613769
,2018-10-10 17:48:34,9.125 sec,400.0,2.6819822,1.2095133,7.1930288
,2018-10-10 17:48:35,9.498 sec,500.0,2.3038664,0.9097620,5.3078003
,2018-10-10 17:48:35,9.927 sec,600.0,2.0890664,0.7690812,4.3641986
,2018-10-10 17:48:36,10.386 sec,700.0,1.9110703,0.6564965,3.6521895
,2018-10-10 17:48:36,10.922 sec,800.0,1.7659607,0.5880838,3.1186170
,2018-10-10 17:48:37,11.466 sec,900.0,1.6192433,0.5239608,2.6219488


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
reconstr_sugars,614.0,1.0,0.0957729
reconstr_fiber,473.0,0.7703583,0.0737794
reconstr_type.C,413.0,0.6726384,0.0644205
reconstr_calories,390.0,0.6351792,0.0608329
reconstr_fat,387.0,0.6302932,0.0603650
---,---,---,---
reconstr_mfr.missing(NA),162.0,0.2638436,0.0252691
reconstr_cups,158.0,0.2573290,0.0246451
reconstr_mfr.P,151.0,0.2459283,0.0235533
reconstr_mfr.K,131.0,0.2133550,0.0204336



See the whole table with table.as_data_frame()


<bound method ModelBase.summary of >